In [ ]:
#imports 
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
from numpy import argmax
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from matplotlib import pyplot

# Leitura dos dados

## Tweets

In [ ]:
#entrada para os dados de treino dos tweets
train_tweets = pd.read_csv('../input/d/guilhermeguimaraes12/dataset/train_tweets.csv')
train_tweets_vectorized_media = pd.read_csv('../input/d/guilhermeguimaraes12/dataset/train_tweets_vectorized_media.csv')
train_tweets_vectorized_text = pd.read_csv('../input/d/guilhermeguimaraes12/dataset/train_tweets_vectorized_text.csv')

In [ ]:
#entrada para os dados de teste dos tweets
test_tweets = pd.read_csv('../input/d/guilhermeguimaraes12/dataset/test_tweets.csv')
test_tweets_vectorized_media = pd.read_csv('../input/d/guilhermeguimaraes12/dataset/test_tweets_vectorized_media.csv')
test_tweets_vectorized_text = pd.read_csv('../input/d/guilhermeguimaraes12/dataset/test_tweets_vectorized_text.csv')

## Users

In [ ]:
#entrada para os dados de user
user_vectorized_descriptions = pd.read_csv('../input/d/guilhermeguimaraes12/dataset/user_vectorized_descriptions.csv')
user_vectorized_profile_images = pd.read_csv('../input/d/guilhermeguimaraes12/dataset/user_vectorized_profile_images.csv')
users = pd.read_csv('../input/d/guilhermeguimaraes12/dataset/users.csv')

# Análise do Target

In [ ]:
train_tweets.virality.value_counts()

In [ ]:
sns.set_style("dark")
sns.countplot(data = train_tweets, x = 'virality')

A coluna Virality é destribuida entre as 5 categorias:
- 1 com 13632 tweets
- 2 com  8889 tweets
- 3 com  4646 tweets
- 4 com  1135 tweets
- 5 com  1323 tweets

## Transformando as features 

### Train

In [ ]:
train_tweets.head()

In [ ]:
#passando as informações da coluna tweet_attachment_class para novas colunas
one_hot = pd.get_dummies(train_tweets['tweet_attachment_class'])
train_tweets = train_tweets.drop('tweet_attachment_class',axis = 1)
train_tweets = train_tweets.join(one_hot)
train_tweets.head()

In [ ]:
#passando as informações da coluna tweet_has_attachment para novas colunas
one_hot = pd.get_dummies(train_tweets['tweet_has_attachment'])
train_tweets = train_tweets.drop('tweet_has_attachment',axis = 1)
train_tweets = train_tweets.join(one_hot)
train_tweets.head()

In [ ]:
train_tweets = train_tweets.drop('tweet_topic_ids',axis = 1)

In [ ]:
c= train_tweets[['tweet_id', 'tweet_user_id', 'virality']]

In [ ]:
train_tweets = train_tweets.drop(['tweet_id', 'tweet_user_id', 'virality'],axis = 1)

In [ ]:
train_tweets.head()

In [ ]:
#normalizando as features
x = StandardScaler().fit_transform(train_tweets) 

In [ ]:
col = train_tweets.columns

In [ ]:
normalised_train = pd.DataFrame(x,columns=col)

In [ ]:
train_tweets_ = pd.concat([c , normalised_train], axis=1)

In [ ]:
train_tweets_

### Test

In [ ]:
#passando as informações da coluna tweet_has_attachment para novas colunas
one_hot = pd.get_dummies(test_tweets['tweet_has_attachment'])
test_tweets = test_tweets.drop('tweet_has_attachment',axis = 1)
test_tweets = test_tweets.join(one_hot)
test_tweets.head()

In [ ]:
##passando as informações da coluna tweet_attachment_class para novas colunas
one_hot = pd.get_dummies(test_tweets['tweet_attachment_class'])
test_tweets = test_tweets.drop('tweet_attachment_class',axis = 1)
test_tweets = test_tweets.join(one_hot)
test_tweets.head()

In [ ]:
test_tweets = test_tweets.drop('tweet_topic_ids',axis = 1)

In [ ]:
c= test_tweets[['tweet_id', 'tweet_user_id']]

In [ ]:
test_tweets = test_tweets.drop(['tweet_id', 'tweet_user_id'],axis = 1)

In [ ]:
test_tweets.head()

In [ ]:
#normalizando as features
x = StandardScaler().fit_transform(test_tweets)

In [ ]:
col = test_tweets.columns

In [ ]:
normalised_test = pd.DataFrame(x,columns=col)

In [ ]:
test_tweets_ = pd.concat([c , normalised_test], axis=1)

In [ ]:
test_tweets_

### User

In [ ]:
users.head()

In [ ]:
##passando as informações da coluna user_has_url para novas colunas
one_hot = pd.get_dummies(users['user_has_url'])
users = users.drop('user_has_url',axis = 1)
users = users.join(one_hot)
users.head()

In [ ]:
users = users.drop('user_has_location',axis = 1)

In [ ]:
users.head()

In [ ]:
c= users[['user_id']]

In [ ]:
users = users.drop(['user_id'],axis = 1)

In [ ]:
users.head()

In [ ]:
#normalizando as features
x = StandardScaler().fit_transform(users) 

In [ ]:
col = users.columns

In [ ]:
normalised_test = pd.DataFrame(x,columns=col)

In [ ]:
users_ = pd.concat([c , normalised_test], axis=1)

In [ ]:
users_.head()

## Unificando o dataset

Estamos juntando os bancos de dados com a função merge a fim de termos uma análise mais direta e prática.

### Train

In [ ]:
train_model = pd.merge(train_tweets_, train_tweets_vectorized_media, on = 'tweet_id',how = 'left' )

In [ ]:
train_model = pd.merge(train_model, train_tweets_vectorized_text, on = 'tweet_id', how = 'left' )

In [ ]:
train_model.shape 

### Test

In [ ]:
test_model = pd.merge(test_tweets_, test_tweets_vectorized_media, on = 'tweet_id', how = 'left' )

In [ ]:
test_model = pd.merge(test_model, test_tweets_vectorized_text, on = 'tweet_id', how = 'left' )

In [ ]:
test_model.shape 

### Users

In [ ]:
user_model = pd.merge(users_, user_vectorized_descriptions, on = 'user_id', how = 'left' )

In [ ]:
user_model = pd.merge(user_model, user_vectorized_profile_images, on = 'user_id', how = 'left' )

In [ ]:
user_model.shape

### Train/Test + Users

In [ ]:
train_model = train_model.rename(columns={'tweet_user_id': 'user_id'})

train_model = pd.merge(train_model, users_, on = 'user_id', how = 'left' )
train_model.shape 

In [ ]:
test_model = test_model.rename(columns={'tweet_user_id': 'user_id'})

test_model = pd.merge(test_model, users_, on = 'user_id', how = 'left' )
test_model.shape 

# Análise de Componentes Principais (PCA)

A Análise de Componentes Principais (PCA) é uma técnica que transforma linearmente p variáveis correlacionadas em k variáveis não-correlacionadas (onde k<p) que explicam parte substancial das informações originais.

Os objetivos principais são:

* 1) Redução da dimensionalidade dos dados sem alterar a variação;
* 2) Obter combinações interpretáveis das variáveis originais;
* 3) Descrever e compreender a estrutura de correlação das variáveis originais.

Decidimos utilizar esse método devido ao grande volume de dados. 

In [ ]:
train_model.head()

Preenchendo NA's com 0


In [ ]:
train_model = train_model.fillna(0)

In [ ]:
test_model = test_model.fillna(0)
test_model.shape

In [ ]:
X = train_model.drop(['tweet_id', 'user_id','media_id', 'virality'],axis = 1)
y = train_model.virality
X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=12,stratify=y)

In [ ]:
#fazendo uma instância do modelo
pca = PCA(.95)

In [ ]:
pca.fit(X_train)

In [ ]:
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
pca.n_components_

# Modelo

Por fim utilizamos a árvore de decisão para fazer a predição no nosso modelo. A ávore de decisão é um método não-paramétrico de aprendizado supervisionado usado para regressão e classificação. 
O objetivo é a predição do valor de uma variável target. Neste caso, nosso target é a variável 'virality'. Ela possui 5 categorias que vão de 1 a 5 medindo o nível de viralidade que o tweet possui.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [ ]:
DecisionTree = DecisionTreeClassifier(random_state = 9847522)

In [ ]:
DecisionTree.fit(X_train, y_train)

In [ ]:
y_pred = DecisionTree.predict(X_test)

In [ ]:
DecisionTree.score(X_test, y_test)

In [ ]:
#Tirando duplicatas
test_ = test_model.drop_duplicates(subset=["tweet_id"])
test_.shape

In [ ]:
id = test_.tweet_id
test_ = test_.drop(['tweet_id', 'user_id','media_id'],axis = 1)

In [ ]:
test_.shape

In [ ]:
test_ = pca.transform(test_)

In [ ]:
y_pred_teste= DecisionTree.predict(test_)

In [ ]:
#Gerando o .CSV da resposta final
resposta = pd.DataFrame(data = {"tweet_id": id, "virality": y_pred_teste})
resposta.to_csv("content\resposta_Task05.csv", index = False)
resposta.head()